### This notebook is just for subsetting the ovarian and skin datasets for malignant cells

In [1]:
import sys
from pathlib import Path

import pandas as pd
import scanpy as sc
from scipy import sparse

sys.path.append("../..")

from constants import BASE_PATH_RAW_CANCER, BASE_PATH_DATA, BASE_PATH_ANNOT_CANCER, BASE_PATH_CANSIG_PP_CANCER
from load_data_3ca import DATA3CA

In [2]:
SAVE = False

#### Ovarian cancer dataset
Vázquez-García, I., Uhlitz, F., Ceglia, N. et al. Ovarian cancer mutational processes drive site-specific immune evasion. Nature 612, 778–786 (2022). https://doi.org/10.1038/s41586-022-05496-1

In [ ]:
adata = sc.read_h5ad(Path(BASE_PATH_RAW_CANCER) / 'OV_Vazquez_10X.h5ad')
adata.shape

In [ ]:
## Filter only for cancer cells
adata = adata[adata.obs['cell_type'] == 'Ovarian.cancer.cell']
adata.shape

In [ ]:
adata = adata[~adata.obs['cluster_label'].isna()]
adata.shape

In [ ]:
adata = adata[~adata.obs['cluster_label'].str.startswith('Ciliated.cell')].copy()
adata.shape

In [ ]:
adata.X = sparse.csr_matrix(adata.X)

In [ ]:
if SAVE:
    # adata.write(Path(BASE_PATH_RAW_CANCER) / 'ovarian_malignant.h5ad')

Data downloaded from [cellxgene](https://cellxgene.cziscience.com/collections/4796c91c-9d8f-4692-be43-347b1727f9d8). This contains the malignant and non-malignant epithelial cells from the same study. However, the data is already curated and preprocessed. We take the cell names from the curated data subset it from the previous data.

In [ ]:
tmp_adata = sc.read_h5ad(Path(BASE_PATH_RAW_CANCER) / 'ovarian_vazquez_cellxgene_epithelial.h5ad')
tmp_adata.shape

In [ ]:
tmp_adata = tmp_adata[~tmp_adata.obs['cluster_label'].str.startswith('Ciliated.cell')].copy()
tmp_adata.shape

In [ ]:
print('Nr. cells in both datasets:\t\t', len(set(tmp_adata.obs_names) & set(adata.obs_names)))
print('Nr. cells only in the curated dataset:\t', len(set(tmp_adata.obs_names) - set(adata.obs_names)))
print('Nr. cells only in the filtered dataset:\t', len(set(adata.obs_names) - set(tmp_adata.obs_names)))

In [ ]:
adata = adata[adata.obs_names.isin(tmp_adata.obs_names)].copy()

In [ ]:
if SAVE:
    # adata.write(Path(BASE_PATH_RAW_CANCER) / 'ovarian_malignant_cellxgene.h5ad')

In [ ]:
del adata

#### Ji et al. 2020 skin dataset
Ji AL, Rubin AJ, Thrane K, Jiang S, Reynolds DL, Meyers RM, Guo MG, George BM, Mollbrink A, Bergenstråhle J, Larsson L, Bai Y, Zhu B, Bhaduri A, Meyers JM, Rovira-Clavé X, Hollmig ST, Aasi SZ, Nolan GP, Lundeberg J, Khavari PA. Multimodal Analysis of Composition and Spatial Architecture in Human Squamous Cell Carcinoma. Cell. 2020 Jul 23;182(2):497-514.e22. doi: 10.1016/j.cell.2020.05.039. Epub 2020 Jun 23. Erratum in: Cell. 2020 Sep 17;182(6):1661-1662. doi: 10.1016/j.cell.2020.08.043. PMID: 32579974; PMCID: PMC7391009.

In [ ]:
def load_skin():
    adata = sc.read(
        Path(BASE_PATH_RAW_CANCER) / 'Ji_et_al_2020' / 'GSE144236_cSCC_counts.txt',
        delimiter='\t')

    metadata = pd.read_table(
        Path(BASE_PATH_RAW_CANCER) / 'Ji_et_al_2020' / 'GSE144236_patient_metadata_new.txt',
        delimiter='\t')

    adata = adata.transpose()
    adata.obs[metadata.columns.to_list()] = metadata.copy()
    adata = adata[:, 2:].copy()
    return adata

In [ ]:
adata = load_skin()

In [ ]:
adata = adata[adata.obs['level2_celltype'].isin(['TSK', 'Tumor_KC_Basal', 'Tumor_KC_Cyc', 'Tumor_KC_Diff'])].copy()
adata.obs.level2_celltype.value_counts()

In [ ]:
adata.X = sparse.csr_matrix(adata.X)

In [ ]:
if SAVE:
    # adata.write(Path(BASE_PATH_RAW_CANCER) / 'skin_malignant_manual.h5ad')

##### Look at the second skin example

In [ ]:
data_path = Path(BASE_PATH_DATA) / 'data_from_florian' / 'data' / 'skin_ji.h5ad'

In [ ]:
adata = sc.read_h5ad(data_path)

In [ ]:
adata = adata[adata.obs['level2_celltype'].isin(['TSK', 'Tumor_KC_Basal', 'Tumor_KC_Cyc', 'Tumor_KC_Diff'])].copy()
adata.obs.level2_celltype.value_counts()

In [ ]:
if SAVE:
    # adata.write(Path(BASE_PATH_CANSIG_PP_CANCER) / 'skin_malignant.h5ad')

#### Kim et al. lung dataset
Kim, N., Kim, H.K., Lee, K. et al. Single-cell RNA sequencing demonstrates the molecular and cellular reprogramming of metastatic lung adenocarcinoma. Nat Commun 11, 2285 (2020). https://doi.org/10.1038/s41467-020-16164-1

In [ ]:
adata = sc.read_h5ad(Path(BASE_PATH_DATA) / 'data_from_florian' / 'data' / 'kim_lung.h5ad')

In [ ]:
samples_in_adata = adata.obs.index.to_list()

In [ ]:
samples_in_adata = adata.obs.index.to_list()
cell_labels = pd.read_table(Path(BASE_PATH_ANNOT_CANCER) / 'luad_kim' / "GSE131907_Lung_Cancer_cell_annotation.txt")
cell_labels['Index'] = cell_labels['Index'].str.replace('_', '-')
cell_labels = cell_labels.set_index('Index')
cell_labels = cell_labels.loc[samples_in_adata]

In [ ]:
y_true_col = 'Cell_subtype'
sample_col = 'sample'

In [ ]:
adata.obs[y_true_col] = cell_labels[y_true_col].str.lower()

In [ ]:
adata = adata[adata.obs[adata.obs[y_true_col].str.startswith('ts', na=False)].index].copy()

In [ ]:
adata.obs[y_true_col].value_counts().sort_index()

In [ ]:
if SAVE:
    # adata.write(Path(BASE_PATH_CANSIG_PP_CANCER) / 'luad_kim_malignant.h5ad')

Sencond version of luad kim directly downloaded from 3CA

In [3]:
data_folder = Path(BASE_PATH_RAW_CANCER) / "luad_kim_2020/Data_Kim2020_Lung"
data_folder

PosixPath('/Users/lciernik/Documents/TUB/projects/ans_scoring/reproduce_project/data/raw_data/luad_kim_2020/Data_Kim2020_Lung')

In [4]:
ds_obj = DATA3CA(data_folder)

In [5]:
ds_obj.validate_data_consistency()

In [6]:
adata_2 = sc.AnnData(
    X=ds_obj.counts,
    obs=ds_obj.cells.reset_index().merge(ds_obj.metadata, on='sample').set_index('cell_name'),
    var=ds_obj.genes
)

In [7]:
adata_2.X

<32493x20793 sparse matrix of type '<class 'numpy.int64'>'
	with 72003915 stored elements in Compressed Sparse Row format>

In [8]:
# Only keep malignant cells
adata_2 = adata_2[adata_2.obs['cell_type'] == 'Malignant'].copy()

In [9]:
samples_in_adata = adata_2.obs.index.to_list()
cell_labels = pd.read_table(Path(BASE_PATH_ANNOT_CANCER) / 'luad_kim' / "GSE131907_Lung_Cancer_cell_annotation.txt")
cell_labels = cell_labels.set_index('Index')
cell_labels = cell_labels.loc[samples_in_adata, :]

In [10]:
y_true_col = 'Cell_subtype'
sample_col = 'sample'
adata_2.obs[y_true_col] = cell_labels[y_true_col].str.lower()

In [11]:
adata_2 = adata_2[adata_2.obs[adata_2.obs[y_true_col].str.startswith('ts', na=False)].index].copy()

In [ ]:
if SAVE:
    adata.write(Path(BASE_PATH_CANSIG_PP_CANCER) / 'luad_kim_malignant_3ca.h5ad')